# Traffic stats

In [ ]:
import pprint, math, datetime, os, copy
from time import strftime, gmtime
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter

plt.rcParams["figure.figsize"] = 30, 20
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

scenarios = [
    "../simulation/performance/traci-n",
    "../simulation/performance/libsumo-n",
    "../simulation/performance/traci-meso-n",
    "../simulation/performance/libsumo-meso-n",
    "../simulation/performance/traci-rr",
    "../simulation/performance/libsumo-rr",
    "../simulation/performance/traci-meso-rr",
    "../simulation/performance/libsumo-meso-rr",
]


In [ ]:
def get_totals(scenario):
    totals = {
        "trips": 0,
        "persons": 0,
        "avg_trips_pp": 0,
        "distance": 0,
        "avg_dist_pt": 0,
        "avg_dist_pp": 0,
        "duration": 0,
        "avg_dur_pt": 0,
        "avg_dur_pp": 0,
        "steps": 0,
        "sim_time": 0
    }

    trips_info_file = f"{scenario}/output/trips-info.xml"
    persons = []
    for event, element in etree.iterparse(trips_info_file, tag="tripinfo"):
        totals["trips"] += 1
        totals["distance"] += float(element.attrib["routeLength"])
        totals["duration"] += float(element.attrib["duration"])

        person, counter = element.attrib["id"].split("_")
        if person not in persons:
            totals["persons"] += 1
            persons.append(person)

    totals["avg_trips_pp"] = totals["trips"] / totals["persons"]

    totals["avg_dist_pt"] = totals["distance"] / totals["trips"]
    totals["avg_dist_pp"] = totals["distance"] / totals["persons"]

    totals["avg_dur_pt"] = totals["duration"] / totals["persons"]
    totals["avg_dur_pp"] = totals["duration"] / totals["persons"]

    start_step = -1
    end_step = -1
    start_sim_time = -1
    end_sim_time = -1
    vehicle_summary_file = f"{scenario}/output/vehicle-summary.xml"
    for event, element in etree.iterparse(vehicle_summary_file, tag="step"):
        if start_step == -1:
            start_step = float(element.attrib["time"])
        else:
            end_step = float(element.attrib["time"])

        if start_sim_time == -1:
            start_sim_time = float(element.attrib["duration"])
        else:
            end_sim_time = float(element.attrib["duration"])

    totals["steps"] = end_step - start_step
    totals["sim_time"] = end_sim_time - start_sim_time

    return totals

totals = get_totals(scenarios[0])
pprint.pprint(totals)

In [ ]:
# Distances in every zone for each trip
def get_distances_pt(scenario, freeze_zones=False):
    file_path = f"{scenario}/output/vehicle-zone-tracking.xml"
    distances_per_zone = {}
  
    for event, element in etree.iterparse(file_path, tag="timestep"):
        current_zone_timestep = element.attrib["zone-timestep"]

        for vehicle in element.getchildren():
            vid = vehicle.attrib["id"]
            speed = float(vehicle.attrib["speed"])
            v_timestep = vehicle.attrib["zone-timestep"]

            distances = {}
            if vid in distances_per_zone:
                distances = distances_per_zone[vid]

            for polygon in vehicle.getchildren():
                pid_parts = polygon.attrib["id"].split("_")
                pid = pid_parts[0]
                p_timestep = pid_parts[1]

                if freeze_zones:
                    if p_timestep != v_timestep:
                        continue
                else:
                    if p_timestep != current_zone_timestep:
                        continue

                if pid.startswith("hole"):
                    hole, hole_counter, zone, polygon_counter = pid.split("-")
                    if zone in distances:
                        distances[zone] -= speed
                    else:
                        distances[zone] = -speed
                else:
                    zone, polygon_counter = pid.split("-")
                    if zone in distances:
                        distances[zone] += speed
                    else:
                        distances[zone] = speed
            
            distances_per_zone[vid] = distances

        element.clear()

    return distances_per_zone

# distances_pt = get_distances_pt(scenarios[0])
# pprint.pprint(distances_pt)

In [ ]:
def get_distances_pp(distances_pt):
    distances_pp = {}

    for trip in distances_pt:
        person, counter = trip.split("_")
        trip_distances = distances_pt[trip]
        
        if person in distances_pp:
            person_distances = distances_pp[person]

            for zone in trip_distances:
                if zone in person_distances:
                    person_distances[zone] += trip_distances[zone]
                else:
                    person_distances[zone] = trip_distances[zone]
                    
            distances_pp[person] = person_distances

        else:
            distances_pp[person] = copy.deepcopy(trip_distances)
    
    return distances_pp

# distances_pt = get_distances_pt(scenarios[0])
# distances_pp = get_distances_pp(distances_pt)
# pprint.pprint(distances_pp)

# x = list(distances_pp.keys())[25]
# print(x)
# pprint.pprint(distances_pp[x])
# print()
# for t in distances_pt:
#     if t.startswith(x):
#         pprint.pprint(distances_pt[t])

In [ ]:
def get_trips_pz(distances_pt):
    trips_pz = {}

    for trip in distances_pt:
        distances_pz = distances_pt[trip]
        for zone in distances_pz:
            if zone in trips_pz:
                trips_pz[zone] += 1
            else:
                trips_pz[zone] = 1
    
    return trips_pz

# distances_pt = get_distances_pt(scenarios[0])
# trips_pz = get_trips_pz(distances_pt)
# pprint.pprint(trips_pz)

### Total persons per zone

In [ ]:
def get_persons_pz(distances_pp):
    persons_pz = {}

    for person in distances_pp:
        distances = distances_pp[person]
        for zone in distances:
            if zone in persons_pz:
                if person not in persons_pz[zone]:
                    persons_pz[zone].append(person)
            else:
                persons_pz[zone] = [person]

    for zone in persons_pz:
        persons_pz[zone] = len(persons_pz[zone])

    return persons_pz

# distances_pt = get_distances_pt(scenarios[0])
# distances_pp = get_distances_pp(distances_pt)
# persons_pz = get_persons_pz(distances_pp)
# pprint.pprint(persons_pz)

In [ ]:
def get_distances_pz(distances_pt):
    distances_pz = {}

    for trip in distances_pt:
        trip_distances = distances_pt[trip]
        for zone in trip_distances:
            if zone in distances_pz:
                distances_pz[zone] += trip_distances[zone]
            else:
                distances_pz[zone] = trip_distances[zone]

    return distances_pz

# distances_pt = get_distances_pt(scenarios[0])
# distances_pz = get_distances_pz(distances_pt)
# pprint.pprint(distances_pz)

In [ ]:
def get_avg_distances_pz_pt(distances_pt):
    avg_distances_pz_pt = {}

    for trip in distances_pt:
        distances_pz = distances_pt[trip]
        for zone in distances_pz:
            if zone in avg_distances_pz_pt:
                avg_distances_pz_pt[zone]["distance"] += distances_pz[zone]
                avg_distances_pz_pt[zone]["n"] += 1
            else:
                avg_distances_pz_pt[zone] = {
                    "distance": distances_pz[zone],
                    "n": 1
                }

    for zone in avg_distances_pz_pt:
        avg_distances_pz_pt[zone] = avg_distances_pz_pt[zone]["distance"] / avg_distances_pz_pt[zone]["n"]
    
    return avg_distances_pz_pt

# distances_pt = get_distances_pt(scenarios[0])
# avg_distances_pz_pt = get_avg_distances_pz_pt(distances_pt)
# pprint.pprint(avg_distances_pz_pt)

In [ ]:
def get_avg_distances_pz_pp(distances_pp):

    avg_distances_pz_pp = {}

    for person in distances_pp:
        person_distances = distances_pp[person]
        for zone in person_distances:
            if zone in avg_distances_pz_pp:
                avg_distances_pz_pp[zone]["distance"] += person_distances[zone]
                avg_distances_pz_pp[zone]["n"] += 1
            else:
                avg_distances_pz_pp[zone] = {
                    "distance": person_distances[zone],
                    "n": 1
                }

    for zone in avg_distances_pz_pp:
        avg_distances_pz_pp[zone] = avg_distances_pz_pp[zone]["distance"] / avg_distances_pz_pp[zone]["n"]

    return avg_distances_pz_pp

# distances_pt = get_distances_pt(scenarios[0])
# distances_pp = get_distances_pp(distances_pt)
# avg_distances_pz_pp = get_avg_distances_pz_pp(distances_pp)
# pprint.pprint(avg_distances_pz_pp)

In [ ]:
wb = Workbook()
ws = wb.active
write_header = True

header = []
row = []

def add(label, value):
    # print(label)
    # print(value)

    if write_header:
        header.append(label)

    row.append(value)

def stringify_zone_data(data, div_by=1, unit=""):
    s = ""
    for zone in data:
        value = data[zone]
        if isinstance(value, float):
            value = format(value / div_by, ".3f")
        s += f"Zone {zone}: {value} {unit}\n"
    return s

for s in scenarios:
    header = []
    row = []

    name = os.path.basename(s)
    print(name)
    add("Scenario", name)

    totals = get_totals(s)
    add("Total steps", totals["steps"])
    add("Total simulation time", f"{format(totals['sim_time'] / 1000, '.3f')} s")

    add("Total trips", totals["trips"])
    add("Total persons", totals["persons"])
    add("Average trips per person", f"{format(totals['avg_trips_pp'], '.3f')}")

    add("Total driven distance", f"{format(totals['distance'] / 1000, '.3f')} km")
    add("Average distance per trip", f"{format(totals['avg_dist_pt'] / 1000, '.3f')} km")
    add("Average distance per person", f"{format(totals['avg_dist_pp'] / 1000, '.3f')} km")

    add("Total driven duration", f"{format(totals['duration'] / 60, '.3f')} min")
    add("Average duration per trip", f"{format(totals['avg_dur_pt'] / 60, '.3f')} min")
    add("Average duration per person", f"{format(totals['avg_dur_pp'] / 60, '.3f')} min")
    
    distances_pt = get_distances_pt(s)
    distances_pp = get_distances_pp(distances_pt)

    trips_pz = get_trips_pz(distances_pt)
    s = stringify_zone_data(trips_pz)
    add("Total trips per zone", s)

    persons_pz = get_persons_pz(distances_pp)
    s = stringify_zone_data(persons_pz)
    add("Total persons per zone", s)

    distances_pz = get_distances_pz(distances_pp)
    s = stringify_zone_data(distances_pz, div_by=1000, unit="km")
    add("Total distances per zone", s)

    avg_distances_pz_pt = get_avg_distances_pz_pt(distances_pt)
    s = stringify_zone_data(avg_distances_pz_pt, div_by=1000, unit="km")
    add("Average distance per zone per trip", s)

    avg_distances_pz_pp = get_avg_distances_pz_pp(distances_pp)
    s = stringify_zone_data(avg_distances_pz_pp, div_by=1000, unit="km")
    add("Average distance per zone per person", s)


    if write_header:
        ws.append(header)
        write_header = False

    
    ws.append(row)   
        
dims = {}
for row in ws.rows:
    for cell in row:
        cell.alignment = Alignment(wrapText=True, vertical="top", horizontal="left") 
        if cell.value:
            dims[cell.column_letter] = max((dims.get(cell.column_letter, 0), len(str(cell.value)))) 

for col, value in dims.items():
    ws.column_dimensions[col].width = value

wb.save("traffic-stats.xlsx")
